## Read cohorts workbook
Assumptions:
 - First column is an id, coding the type of cohort. Starts with T for target, O for outcome or S for strata.
 - Second column is the cohort name
 - A column exists with the link to the cohort id on PIONEER Atlas

In [18]:
import xlrd
import re

In [19]:
book = xlrd.open_workbook('OHDSI studyathon phenotype tracker APRIL 2021.xlsx')

In [20]:
sheet = book.sheet_by_index(0)

In [21]:
PHENOTYPE_ID_PATTERN = re.compile(r'([TOS])(.+)')
COHORT_LINK_PATTERN = re.compile(r'/cohortdefinition/(\d+)')

In [22]:
def find_cohort_atlas_id(row):
    for col in row:
        if not col.value:
            continue
        cohort_link_match = COHORT_LINK_PATTERN.search(col.value)
        if cohort_link_match:
            return cohort_link_match.group(1)

In [23]:
# To keep the old numbering
id_lookup = {
    'O1': 202,
    'O2': 201,
    'S31': 328,
    'S32': 329,
    'S33': 330,
    'S34': 331,
    'S35': 332,
    'S36': 333,
    'S28': 334,
    'S29': 335,
    'S30': 336,
    'T1a': 102,
    'T2' : 103,
    'T2a': 104,
    'T3.1': 105,
    'T3.1a': 106,
    'T3.2': 107,
    'T3.2a': 108,
    'T3.3': 109,
    'T3.3a': 110,
    'T3': 111,
    'T3a': 112,
    'T4.1': 113,
    'T4.1a': 114,
    'T4.2': 115,
    'T4.2a': 116,
    'T4': 117,
    'T4a': 118
}

In [24]:
cohorts = {
    'T': [],
    'O': [],
    'S': []
}
for rx in range(sheet.nrows):
    row = sheet.row(rx)
    id_match = PHENOTYPE_ID_PATTERN.search(row[0].value)
    if not id_match:
        continue

    cohort_code = id_match.group(0)
    cohort_type = id_match.group(1)
    #cohort_name = f'{row[1].value}'
    cohort_atlas_id = find_cohort_atlas_id(row)
    cohort_atlas_name = f'[PIONEER {cohort_code}] {row[1].value}'
    
    if cohort_code in id_lookup:
        cohort_id = id_lookup.get(cohort_code)
    elif cohort_type == 'T':
        cohort_id = 101 + len(cohorts['T'])
    elif cohort_type == 'O':
        cohort_id = 201 + len(cohorts['O'])
    elif cohort_type == 'S':
        cohort_id = 301 + len(cohorts['S'])
        
    if cohort_atlas_id is None:
        print(f'Skipped Cohort "{cohort_code:3.3}": No Atlas link found')
        continue

    cohorts[cohort_type].append({
        'name': cohort_id,  # insertCohortDefinitionSetInPackage names cohort files according to this 
        'atlasName': cohort_atlas_name,
        'atlasId': cohort_atlas_id,
        'cohortId': cohort_id  # instantiateCohortSet uses the cohort id to find the file
    })

In [25]:
[(k,len(v)) for k,v in cohorts.items()]

[('T', 20), ('O', 7), ('S', 38)]

## Write cohorts csv
Write the 'CohortsToCreate' csv files for target, outcome and strata. 
Repeat for diagnostics.

In [26]:
import csv

In [27]:
def write_cohorts_to_create(filename, rows):
    with open(filename, 'w') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=['name', 'atlasName', 'atlasId', 'cohortId'])
        writer.writeheader()
        writer.writerows(rows)

In [28]:
write_cohorts_to_create('settings/CohortsToCreateTarget.csv', cohorts['T'])

In [29]:
write_cohorts_to_create('settings/CohortsToCreateOutcome.csv', cohorts['O'])

In [30]:
write_cohorts_to_create('settings/CohortsToCreateStrata.csv', cohorts['S'])

In [31]:
write_cohorts_to_create('settings/diagnostics/CohortsToCreateTarget.csv', cohorts['T'])

In [32]:
write_cohorts_to_create('settings/diagnostics/CohortsToCreateOutcome.csv', cohorts['O'])

In [33]:
write_cohorts_to_create('settings//diagnostics/CohortsToCreateStrata.csv', cohorts['S'])